Copyright: Vrije Universiteit Amsterdam, Faculty of Humanities, CLTL

# Lab 1.3: Extracting Metadata

In this notebook, we use the MediaStack to search for news: 

https://mediastack.com/ 

We learn how to extract some metadata from the API. 

Go to https://mediastack.com/signup and fill in the required information.
<br>
Save the API Access Key 

## 1. Queries in different languages

In the Media Stack API, we can vary different parameters, such as the keywords and the language.

The language needs to be abbreviated according to the two-letter ISO-639-1 code.

Play with different queries and languages.

Note: There are different ISO code classifications for languages. ISO-639-1 is the oldest one and uses two letters. More recent schemes use three letters to include more languages (living and extinct):

https://www.iso.org/iso-639-language-codes.html
https://en.wikipedia.org/wiki/List_of_ISO_639-1_codes

In [3]:
import http.client, urllib.parse, json
from util_html import *

conn = http.client.HTTPConnection('api.mediastack.com')

params = urllib.parse.urlencode({
    'access_key': , ## ADD YOUR ACCESS KEY
    'keywords' : 'veganism',
    'sort': 'published_desc',
    'languages':'en'
    })

conn.request('GET', '/v1/news?{}'.format(params))

res = conn.getresponse()
data = res.read()

query_content=(data.decode('utf-8'))
query = json.loads(query_content)

# 2. Extracting Metadata

Media Stack lists many articles for each query. As with the NOS-articles, we first want to extract the links. If you click on the links, you will notice that Media Stack does not write own articles, but just lists articles from other sources. In the following function, we try to extract metadata from the html. 

This particular strategy for metadata extraction only works for this version of Media Stack.
If you use another engine or if their code changes, you will need to adapt the metadata extraction. 

**Make sure to add additional printouts to inspect the html content and understand how we find the metadata.** 

In [6]:
def extract_metadata(article):
    # Extract the publication date
    published_at = article['published_at']
    if published_at:
        date, time = published_at.split("T")        
    else:
        date = ""
        time = ""

    # Extract meta data
    url = article ['url']
    title= article['title'] 
    
    # category associated with the given news article
    category = article['category']
    
    # country code associated with given article 
    country = article ['country']
    
    return date, time, title, url, category, country
    

# 3 Extracting Content

In util_html.py, you find two additional functions: *parse_author* and *parse_news_text*. These functions try to extract the author and the text from each article. Note, that the functions are only approximations. They might fail because we do not know the html structure of every publisher. 

In [7]:
articles = query["data"]

max = 10
for i, article in enumerate(articles):
    if i < max: 
        
        date, time, title, article_url, category, country = extract_metadata(article)
    

        article_content = url_to_html(article_url)   
        author = parse_author(article_content)
        content = parse_news_text(article_content)
        
        print(date, time)
        print(article_url)
        print("author:", author)  
        print("title:", title) 
        print("category:",category)
        print("country:",country)
        print(content[:100])
        print()
    else:
        break

2021-11-01 09:45:30+00:00
https://metro.co.uk/2021/11/01/going-vegan-made-me-afraid-of-losing-connection-to-my-pakistani-heritage-15519706/
author: Anya Ryan
title: Going vegan made me afraid of losing connection to my Pakistani heritage
category: general
country: gb
Growing up, food was a language of cultural connection for me. Having never travelled to Pakistan an

2021-10-31 05:30:19+00:00
https://www.theguardian.com/lifeandstyle/2021/oct/31/the-best-new-fragrances
author: https://www.theguardian.com/profile/funmi-fetto
title: The best new fragrances | Funmi Fetto
category: entertainment
country: gb
We are fast approaching that time of the year when we should all be getting our proverbials in gear 

2021-10-30 10:11:20+00:00
https://www.argophilia.com/news/celebrate-world-vegan-day-with-crumb-healthy-living/230219/
author: 
title: Celebrate World Vegan Day With Crumb Healthy Living
category: general
country: us


2021-10-30 02:55:38+00:00
https://dailytimes.com.pk/835349/riteish-des

# 4. Saving results as TSV

A standardized open format for storing the content of multiple variables are CSV files. CSV stands for comma-separated values. CSV files are text-based, but when they are imported to a spreadsheet program such as Excel, they are displayed as a table. Lines in the text file are interpreted as rows; commas in the text file are interpreted as separators for columns.  

Most programmers prefer to use TSV files. In these files, the values are separated by tabulators ("\t") instead of commas. Both variants can be easily processed, but you need to know which separator has been used. 

**If necessary, recap information on CSV and TSV files in [Chapter 16](https://github.com/cltl/python-for-text-analysis/blob/master/Chapters/Chapter%2016%20-%20Data%20formats%20I%20(CSV%20and%20TSV).ipynb) of the python course.** 

In [8]:
conn = http.client.HTTPConnection('api.mediastack.com')

keywords = 'veganism'

params = urllib.parse.urlencode({
    'access_key': '',## YOUR ACCESS KEY
    'keywords' : keywords,
    'sort': 'published_desc',
    'languages':'en'
    })

conn.request('GET', '/v1/news?{}'.format(params))

res = conn.getresponse()
data = res.read()

query_content=(data.decode('utf-8'))
query = json.loads(query_content)

outfile = "../results/mediastack_results/" + keywords +"_overview.tsv"

with open(outfile, "w",encoding="utf-8") as f:
    date, time, title, article_url, category, country = extract_metadata(article)

    f.write("Publication Date\tTime\tAuthor\tTitle\tURL\tText\n")
    
    for i, article in enumerate(articles):
        
        # Extract metadata
        date, time, title, article_url, category, country = extract_metadata(article)
        
        # Extract content
        article_content = url_to_html(article_url)
        author = parse_author(article_content)
        content = parse_news_text(article_content)
        
        # We remove the newlines from the content, so that we can easily store it in a single line. 
        # Keep in mind, that newlines can also carry meaning.
        # For example, they separate paragraphs and this information is lost in the analysis, if we remove them. 
        content = content.replace("\n", "")
        
        # We want the fields to be separated by tabulators (\t)
        output = "\t".join([date, time, author, title, article_url, content])
        f.write(output +"\n")


# 5. Inspect the results.

You will notice that we do not always find a value for the author. There can be two reasons for that: 
- The author name is not provided by the publisher.
- Our code cannot find it.

Double-check on the website which explanation holds. When we are working with automatic methods, we will always be confronted with the issue of missing data. 

**Discuss how this can affect the methodology and interpretation of your experiments.** 